In [136]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from math import log2, log10, log, e, exp

plt.rcParams["figure.figsize"] = (18, 9)

In [137]:
train_path = "andan-2022-autumn-lab-4/train.csv"
test_path = "andan-2022-autumn-lab-4/test_x.csv"

In [138]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [140]:

population_mean = 2607030.2667519217
towns_with_metro = {
    "Москва", "Санкт-Петербург", "Новосибирск", "Екатеринбург", "Казань", "Нижний Новгород", "Самара"
}
# train_data[train_data["city"] == "Самара"]["osm_subway_closest_dist"].mean() # везде не больше 4

search = [ "per_square_meter_price" ]
params = [
    "city",

    "total_square",
    "osm_train_stop_closest_dist",
    "osm_subway_closest_dist",
    "osm_city_nearest_population", # have NaN of 35 count
    "osm_amenity_points_in_0.01",
    "osm_building_points_in_0.01",
    "osm_catering_points_in_0.01",
    "osm_crossing_points_in_0.01",
    "osm_culture_points_in_0.01",
    "osm_finance_points_in_0.01",
    "osm_healthcare_points_in_0.01",
    "osm_historic_points_in_0.01",
    "osm_hotels_points_in_0.01",
    "osm_leisure_points_in_0.01",
    "osm_offices_points_in_0.01",
    "osm_shops_points_in_0.01",
    "osm_train_stop_points_in_0.01",
    "osm_transport_stop_points_in_0.01",
    "reform_count_of_houses_1000",
]
# new_params = [
#     "is_msk",
#     "is_spb"
# ]

# _dict = dict()
# _train_data = pd.read_csv(train_path)
# _test_data = pd.read_csv(test_path)
# for name in params:
#     if name in [ "city" ]: # ignore
#         continue
#     _dict[name] = (min(min(_train_data[name]), min(_test_data[name])), max(max(_train_data[name]), max(_test_data[name])))


# # FIXME переделать под заранее определенные min и max
# def minmax(col, _min, _max):
#
#     col[col < _min] = 0
#     col[_max < col] = 1
#     # from sklearn.preprocessing import minmax_scale
#     # col[(_min < col) & (col < _max)] = minmax_scale(col[(_min < col) & (col < _max)], copy=False)
#     col[(_min < col) & (col < _max)] = (col[(_min < col) & (col < _max)] - _min) / (_max - _min)
#     return col
#
# def _minmax(val, name):
#
#     _min, _max = _dict[name]
#
#     # from sklearn.preprocessing import minmax_scale
#     # col[(_min < col) & (col < _max)] = minmax_scale(col[(_min < col) & (col < _max)], copy=False)
#     # val[(_min < val) & (val < _max)] = (val[(_min < val) & (val < _max)] - _min) / (_max - _min)
#     return (val - _min) / (_max - _min)

def city_handler(s):
    return [
        1 if s in towns_with_metro else 0, # metro in city
        1 if s == "Москва" else 0,
        1 if s == "Санкт-Петербург" else 0,
    ]

def line_handler(line):
    ma = []
    have_metro = False
    for name in params:
        if name in [
                    # 'per_square_meter_price',
                    ]: # ignore
            continue
        elif name in ['city']:
            ma += city_handler(line[name])
            if line[name] in towns_with_metro:
                have_metro = True
        elif name in ["osm_subway_closest_dist"]:
            ma += [line[name]] if have_metro else [0]
            # ma += [_minmax(line[name], name)] if have_metro else [0]
        else:
            ma += [line[name]]
            # ma += [_minmax(line[name], name)]
    return ma

def getRegression(data, is_train=True):

    data = data[params + (search if is_train else [])]

    # data = data[data.osm_city_nearest_population.notna()]
    data.osm_city_nearest_population.fillna(population_mean, inplace=True)

    X = []
    for i in range(len(data)):
        line = data.iloc[i]
        X += [line_handler(line)]
        # print(X)

    # print(X[0])
    X = np.array(X)
    # print(X[0])
    return X, data['per_square_meter_price'].apply(log) if is_train else None

def unlog(x):
    return exp(x)

In [141]:
reg = LinearRegression(copy_X=False, n_jobs=-1)

In [142]:
train_data_original = pd.read_csv(train_path)
train_data = train_data_original.copy()

In [143]:
X, per_square_meter_price = getRegression(train_data)

C:\Users\Legend\AppData\Local\Temp\ipykernel_6404\1811973803.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.osm_city_nearest_population.fillna(population_mean, inplace=True)


In [144]:
X_with_org_original = list(zip(X, list(per_square_meter_price)))

In [145]:
errors = []
def pred(X, per_square_meter_price, test_x, test_y, reg):
    reg.fit(X, per_square_meter_price)
    prc = list(map(lambda x: x if 0 < x else 0, reg.predict(test_x)))
    errors.append(100 * mean_absolute_percentage_error(list(map(unlog, test_y)), list(map(unlog, prc))))
    print(errors[-1])

for i in range(10):
    import random
    X_with_org = X_with_org_original.copy()
    random.shuffle(X_with_org)
    train_comb = X_with_org[int(len(X_with_org)*0.8):]
    test_comb = X_with_org[:int(len(X_with_org)*0.8)]
    X, per_square_meter_price = zip(*train_comb)
    test_x, test_y = zip(*test_comb)

    pred(X, per_square_meter_price, test_x, test_y, reg)

print("Mean:", sum(errors)/len(errors))
# 1) 308.928935118498
# 2) add start/end to dists
# 299.0469893742038
# 3) remove city
# 395.15472764739865

# 5) without minmax
# 253.5033998987495

# 6) with more params
# 229.1704818166524

# 7) 0 in towns without metro
# 230.49679403505212

# 8) remove negate value
# 212.15129672890285

# 9) log on answer
# 155.1173907344399

153.38576945432686
151.94773467065772
153.53930120248813
155.05033137975033
153.8499997490724
155.55481925575444
153.9394663409877
154.07883684469329
154.93541347273697
156.37549869457396
Mean: 154.26571710650416


### Отсюда пойдут предсказания

In [146]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [147]:
X, per_square_meter_price = getRegression(train_data)

C:\Users\Legend\AppData\Local\Temp\ipykernel_6404\1811973803.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.osm_city_nearest_population.fillna(population_mean, inplace=True)


In [148]:
reg.fit(X, per_square_meter_price)

LinearRegression(copy_X=False, n_jobs=-1)

In [149]:
X_test, _ = getRegression(test_data, False)

C:\Users\Legend\AppData\Local\Temp\ipykernel_6404\1811973803.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.osm_city_nearest_population.fillna(population_mean, inplace=True)


In [150]:
# ma = map(lambda x: x if 0 < x else 0, reg.predict(X_test))
ma = map(unlog, reg.predict(X_test))

In [151]:
from itertools import count
test_y = pd.DataFrame(zip(ma, count(0)), columns=['per_square_meter_price', 'id'])
test_y.to_csv('andan-2022-autumn-lab-4/saved_submission_4lab.csv', index=False)
print(test_y)

       per_square_meter_price     id
0                92658.579564      0
1                28991.982923      1
2                35065.705938      2
3                32503.277503      3
4                23715.963853      4
...                       ...    ...
68820            27626.813711  68820
68821            35408.046116  68821
68822            58798.647219  68822
68823           176453.147429  68823
68824            44002.034956  68824

[68825 rows x 2 columns]
